# Проект: Нейросеть для автодополнения текстов

Сравнение LSTM и distilgpt2 для задачи автодополнения коротких текстовых постов.

**Датасет:** sentiment140 (короткие текстовые посты, ~1.6M записей)

**Задача:** по началу текста (3/4) предсказать продолжение (1/4)

## Этап 1. Сбор и подготовка данных

In [ ]:
import sys
sys.path.insert(0, 'src')

from data_utils import load_raw_data, preprocess_dataset, split_dataset

df = load_raw_data('data/raw_dataset.txt')
print(f'Исходный датасет: {len(df)} строк')

df = preprocess_dataset(df)
print(f'После очистки: {len(df)} строк')
print(df['text'].head(10))

In [ ]:
# Разбиваем и сохраняем
train_df, val_df, test_df = split_dataset(df)
df.to_csv('data/dataset_processed.csv', index=False)
train_df.to_csv('data/train.csv', index=False)
val_df.to_csv('data/val.csv', index=False)
test_df.to_csv('data/test.csv', index=False)

print(f'Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

In [ ]:
from next_token_dataset import create_dataloaders

tokenizer, train_loader, val_loader, test_loader = create_dataloaders(
    'data/train.csv', 'data/val.csv', 'data/test.csv',
    batch_size=128, min_freq=5
)

x_batch, y_batch = next(iter(train_loader))
print(f'Batch X: {x_batch.shape}, Y: {y_batch.shape}')
print(f'Словарь: {tokenizer.vocab_size} слов')

# Пример кодирования/декодирования
sample = 'i love this movie so much'
encoded = tokenizer.encode(sample)
decoded = tokenizer.decode(encoded)
print(f'\nОригинал: {sample}')
print(f'Encoded:  {encoded}')
print(f'Decoded:  {decoded}')

## Этап 2. Реализация LSTM модели

In [ ]:
import torch
from lstm_model import LSTMModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

model = LSTMModel(
    vocab_size=tokenizer.vocab_size,
    embed_dim=64,
    hidden_dim=128,
    num_layers=2,
    dropout=0.3
).to(device)
print(f'Параметры модели: {sum(p.numel() for p in model.parameters()):,}')

# Проверка forward
logits, _ = model(x_batch.to(device))
print(f'Logits shape: {logits.shape}')

# Проверка генерации (необученная модель)
result = model.generate(tokenizer, 'i love this', max_new_tokens=5, device=device)
print(f'\nГенерация (до обучения): {result}')

## Этап 3. Обучение LSTM

In [ ]:
from lstm_train import train_model

history = train_model(
    model, train_loader, val_loader, tokenizer, device,
    epochs=10, lr=0.001, save_path='models/lstm_best.pt'
)

In [ ]:
# Графики обучения
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_title('Loss')
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history['rouge1'])
axes[1].set_title('ROUGE-1')
axes[1].set_xlabel('Epoch')
axes[1].grid(True)

axes[2].plot(history['rouge2'])
axes[2].set_title('ROUGE-2')
axes[2].set_xlabel('Epoch')
axes[2].grid(True)

plt.tight_layout()
plt.savefig('models/training_history.png', dpi=100)
plt.show()

### Оценка LSTM на валидации и тесте

In [ ]:
from lstm_train import compute_rouge

model.load_state_dict(torch.load('models/lstm_best.pt', map_location=device))

print('=== LSTM: Валидация ===')
lstm_r1_val, lstm_r2_val, examples_val = compute_rouge(
    model, val_loader, tokenizer, device, max_samples=500
)
print(f'ROUGE-1: {lstm_r1_val:.4f}')
print(f'ROUGE-2: {lstm_r2_val:.4f}')
print('\nПримеры:')
for ex in examples_val[:5]:
    print(f'  Вход:   {ex["input"]}')
    print(f'  Таргет: {ex["target"]}')
    print(f'  Модель: {ex["generated"]}\n')

print('\n=== LSTM: Тест ===')
lstm_r1_test, lstm_r2_test, examples_test = compute_rouge(
    model, test_loader, tokenizer, device, max_samples=500
)
print(f'ROUGE-1: {lstm_r1_test:.4f}')
print(f'ROUGE-2: {lstm_r2_test:.4f}')
print('\nПримеры:')
for ex in examples_test[:5]:
    print(f'  Вход:   {ex["input"]}')
    print(f'  Таргет: {ex["target"]}')
    print(f'  Модель: {ex["generated"]}\n')

### Примеры свободной генерации LSTM

In [ ]:
prompts = [
    'i love this',
    'going to the',
    'i am so',
    'just got back from',
    'why is everyone',
    'the weather is',
    'i want to',
    'happy birthday',
]
print('Примеры автодополнения (LSTM):\n')
for p in prompts:
    result = model.generate(tokenizer, p, max_new_tokens=8, device=device)
    print(f'  {p} -> {result}')

## Этап 4. Предобученный трансформер (distilgpt2)

In [ ]:
from eval_transformer_pipeline import evaluate_transformer

print('=== distilgpt2: Валидация ===')
gpt_r1_val, gpt_r2_val, gpt_ex_val = evaluate_transformer(
    'data/val.csv', max_samples=500
)
print(f'ROUGE-1: {gpt_r1_val:.4f}')
print(f'ROUGE-2: {gpt_r2_val:.4f}')
print('\nПримеры:')
for ex in gpt_ex_val[:5]:
    print(f'  Вход:   {ex["input"]}')
    print(f'  Таргет: {ex["target"]}')
    print(f'  Модель: {ex["generated"]}\n')

In [ ]:
print('=== distilgpt2: Тест ===')
gpt_r1_test, gpt_r2_test, gpt_ex_test = evaluate_transformer(
    'data/test.csv', max_samples=500
)
print(f'ROUGE-1: {gpt_r1_test:.4f}')
print(f'ROUGE-2: {gpt_r2_test:.4f}')
print('\nПримеры:')
for ex in gpt_ex_test[:5]:
    print(f'  Вход:   {ex["input"]}')
    print(f'  Таргет: {ex["target"]}')
    print(f'  Модель: {ex["generated"]}\n')

## Этап 5. Выводы

In [ ]:
# Сводная таблица метрик
print('=' * 70)
print(f'{"Модель":<15} {"ROUGE-1 val":>12} {"ROUGE-2 val":>12} {"ROUGE-1 test":>13} {"ROUGE-2 test":>13}')
print('-' * 70)
print(f'{"LSTM":<15} {lstm_r1_val:>12.4f} {lstm_r2_val:>12.4f} {lstm_r1_test:>13.4f} {lstm_r2_test:>13.4f}')
print(f'{"distilgpt2":<15} {gpt_r1_val:>12.4f} {gpt_r2_val:>12.4f} {gpt_r1_test:>13.4f} {gpt_r2_test:>13.4f}')
print('=' * 70)

### Анализ

1. **distilgpt2 превосходит LSTM** по обеим метрикам ROUGE на валидационной и тестовой выборках.

2. **Качество генерации**: По примерам видно, что distilgpt2 генерирует более грамматически корректные и семантически связные продолжения. LSTM генерирует условно связные тексты, но хуже попадает в контекст исходной фразы.

3. **Причины разницы**:
   - distilgpt2 предобучена на огромном корпусе текстов и уже «знает» английский язык.
   - LSTM обучалась с нуля на датасете sentiment140.
   - Архитектура трансформера лучше улавливает долгосрочные зависимости в тексте.

4. **Размер моделей**:
   - LSTM: ~15M параметров (embed_dim=64, hidden_dim=128, словарь ~77K слов).
   - distilgpt2: ~82M параметров с subword-токенизацией (BPE), словарь ~50K подслов.

### Рекомендации

Для продуктового использования рекомендуется **distilgpt2**, так как:
- Значительно лучшее качество генерации без необходимости обучения на собственных данных.
- Приемлемый размер модели для мобильных устройств (с оптимизациями: квантизация, ONNX-конвертация).
- Subword-токенизация делает модель устойчивой к опечаткам и новым словам.

LSTM может быть оправдана только при жёстких ограничениях по памяти устройства (<10MB), но потребует значительно больше данных и вычислительных ресурсов для достижения приемлемого качества.